In [9]:
from pylab import *
%matplotlib inline

# Problema 1 

## Función de raíces por bipartición



In [10]:
def raiz_bip(func,x_izq,x_der, err=1e-5): #(la función, punto ancla izquiero, punto ancla derecho, error)
    raiz = 0.5*(x_izq + x_der)
    y_izq, y_der = func(x_izq), func(x_der)
    n,nmax=0,500
    
    while abs(y_der) >=err and n <=nmax:
        y_medio = func(raiz)
        
        #Por si le atina a la raíz... (que es algo raro pero puede suceder)
        if y_medio == 0 :
            print "Le atiné a la raíz" 
            break 
        
        #Si no le atina a la raíz:
        if y_medio*y_izq < 0:
            x_der = raiz
        else:
            x_izq = raiz
        
        raiz = 0.5*(x_izq+x_der)
        y_izq,y_der = func(x_izq),func(x_der)
        n += 1 #Contador
        
    if n >= nmax or abs(func(raiz)) > err:
        return False #"No hay raiz en este intervalo"
    else:
        return raiz

In [11]:
def todas_raices(funcion, intervalo, n , tol = 1e-6):
    x = linspace(intervalo[0],intervalo[1],n)
    raices = []
    
    for i in range(len(x)-1):
        sol=raiz_bip(funcion, x[i], x[i+1],tol)
        if sol != False:
            raices.append(sol)
        
    return raices
        

# Problema 2

In [12]:
def raiz_bip_2d(funcion,r_izq, r_der ,err=1e-5): #siguiendo la misma idea que "raiz_bip"
    r_izq, r_der = array(r_izq) , array(r_der)   #puesto que ahora estamos en dos dimensiones, vamos a hacer
                                                 #un array de los valores en el intervalo
    raiz = 0.5*(r_izq+ r_der)     
    func_izq, func_der = funcion(r_izq) , funcion(r_der)   

    while abs(func_izq - func_der)>= err:
        func_med = funcion(raiz)
        if func_med==.0:
            break
        if func_izq*func_med < 0:
            r_der = raiz
        else:
            r_izq = raiz
        raiz = 0.5 *(r_izq + r_der)
        func_izq , func_der = funcion(r_izq) , funcion(r_der)
    return array(raiz)   

In [13]:
def todas_raices_2d(funcion, intervalo, tol=1e-2):  #donde: intervalo = [x_i,x_f,y_i,y_f]
    x = arange(intervalo[0],intervalo[1]+tol, tol)
    y = arange(intervalo[2],intervalo[3]+tol, tol)
    raices_x = []  #dejamos abierto el tamaño del array porque no sabemos cuantas raices va a encontrar
    raices_y = []
    
    for i in xrange(len(x)-1):      #estos dos for son para barrer primero la funcion en los valores de X y despues
                                    # con uno fijo, barrer los de Y. 
        for j in xrange(len(y)-1):
            r_izq = [x[i], y[j]]
            r_der = [x[i], y[j+1]]
            f_izq, f_der = funcion(r_izq) , funcion(r_der)
            
            if f_izq*f_der<=0:
                raiz = raiz_bip_2d(funcion,r_izq,r_der)
                raices_x.append(raiz[0]) #Guardo los valores en dos listas una para X y otra para Y
                raices_y.append(raiz[1])
                
    return array([raices_x, raices_y])

# Problema 3

Método de Runge-Kutta de 4to orden

In [14]:
def rk4(ec,p_ini,x,args=0,h=0.01):
    if (x[1]-x[0]) <= 5.*h:
        h = (x[1]-x[0])/10
    tiempos = arange(x[0],x[1]+h,h)
    sol = zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    if args==0:
        for i in xrange(len(tiempos)-1):
            k1 = ec(sol[i,:],tiempos[i])
            k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h)
            k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h)
            k4 = ec(sol[i,:]+h*k3,tiempos[i]+h)
            sol[i+1,:]=sol[i,:]+(h/6.)*(k1+2*k2+2*k3+k4)
    else:
        for i in xrange(len(tiempos)-1):
            k1 = ec(sol[i,:],tiempos[i],args)
            k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h,args)
            k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h,args)
            k4 = ec(sol[i,:]+h*k3,tiempos[i]+h,args)
            sol[i+1,:]=sol[i,:]+(h/6.)*(k1+2*k2+2*k3+k4)
    return tiempos,sol

La función auxiliar para graficar y que todo sea mas sencillo: 

In [16]:
def datos_graf(condicion_ini, angulo, modelo_fric=0 , pts = 100, dt=0.1):
    N = pts
     #condicion_ini = [x,y,vx,vy]
    inicio = condicion_ini
    trayectoria = zeros((N,len(inicio)))
    mdt = dt
    trayectoria[0,:] = array(inicio)
    
    for i in range(N-1):
        t , sol_dt = rk4(e_mov,trayectoria[i,:],[0,mdt], modelo_fric)
    
        if sol_dt[-1,:][1] - plano(sol_dt[-1,:][0], angulo) < 0:
            y_izq = rebote(trayectoria[i,:], mdt, angulo)    
            trayectoria[i+1,:] = y_izq
        else:
            trayectoria[i+1,:] = sol_dt[-1,:]
    
    return trayectoria